In [67]:
import sys
import os
import sklearn
import pandas as pd
import numpy as np
import xml
import re
from time import time
import matplotlib.pyplot as plt
from pprint import pprint
import html


print(sys.version)

3.5.5 |Anaconda, Inc.| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]


In [2]:
# Download the datasets if they are not already downloaded
if 'data' not in os.listdir('.'):
    !mkdir './data'
if not os.listdir('./data'):
    !wget https://hotell.difi.no/download/nav/ledige-stillinger/2017?download -O ./data/stillingsannonser.csv
    !wget https://hotell.difi.no/download/nav/stillingstekster/2017?download -O ./data/stillingsannonser-beskrivelser.csv


In [3]:
import wget

data_dir = os.path.join('.', 'data')

years_with_desc = [2002, 2015, 2016, 2017]
years_with_ads = list(range(2002, 2017 + 1))

def download_data(start_year=2012, end_year=2017):
    if 'data' not in os.listdir('.'):
        !mkdir 'data'
    for year in range(start_year, end_year + 1):
        filename_ads = ''.join([str(year), '_ads', '.csv'])
        filename_desc = ''.join([str(year), '_desc', '.csv'])
        ads_url = ''.join(['https://hotell.difi.no/download/nav/ledige-stillinger/', str(year), '?download'])
        desc_url = ''.join(['https://hotell.difi.no/download/nav/stillingstekster/', str(year), '?download'])
        
        if filename_ads not in os.listdir(os.path.join(data_dir)) and year in years_with_ads:
            wget.download(ads_url, out=os.path.join(data_dir, filename_ads))
            print(''.join(['Downloaded ads for ', str(year)]))
        if filename_desc not in os.listdir(os.path.join(data_dir)) and year in years_with_desc:
            wget.download(desc_url, out=os.path.join(data_dir, filename_desc))
            print(''.join(['Downloaded desc for ', str(year)]))

print(years_with_desc)
print(years_with_ads)

[2002, 2015, 2016, 2017]
[2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]


In [4]:
download_data(2012, 2017)

In [5]:
# Load dataset from csv
data_dir_path = os.path.join('.', 'data')
year = 2015
job_ads_filename = ''.join([str(year), '_ads.csv'])
job_ads_desc_filename = ''.join([str(year), '_desc.csv'])

df_ads = pd.read_csv(
    os.path.join(data_dir_path, job_ads_filename),
    sep=';'
)

df_ads_desc = pd.read_csv(
    os.path.join(data_dir_path, job_ads_desc_filename),
    sep=';'
)

# Merge datasets into one
df = pd.merge(df_ads, df_ads_desc)

# Rename columns that have different names in different data sets
df.rename(columns={'stillingsutlysning': 'stillingsbeskrivelse'}, inplace=True)

df.head()

,stillingsnummer,nav_enhet_kode,registrert_dato,sistepubl_dato,statistikk_aar_mnd,offisiell_statistikk_flagg,stilling_kilde,arbeidssted_fylkesnummer,arbeidssted_fylke,arbeidssted_kommunenummer,...,yrke_grovgruppe,yrkeskode,yrke,yrkesbetegnelse,virksomhet_organisasjonsnr,virksomhet_navn,antall_stillinger,stillingstittel,statistikk_periode,stillingsbeskrivelse
0,101201412000010,101,18.12.2014,02.01.2015,201501,1,Annonsert i media,01,Østfold,0101,...,Undervisning,2342,Førskolelærere,Førskolelærer,973967592,HØGSKOLEN I ØSTFOLD,1,Barnehagelærar / Førstekonsulent (14/04241),201501,# senter for framandspråk i opplæringa er det ...
1,101201412000011,101,19.12.2014,01.02.2015,201501,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Serviceyrker og annet arbeid,5413,Fengselsbetjenter,Fengselsbetjent,993315060,HALDEN FENGSEL,1,Halden fengsel - ledig fast stilling som verks...,201501,Jobbnorge ID: 108617<br><br><p dir=#LTR# align...
2,101201412000012,101,19.12.2014,16.01.2015,201501,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Serviceyrker og annet arbeid,5413,Fengselsbetjenter,Fengselsbetjent,993315060,HALDEN FENGSEL,1,Halden fengsel - ledig fast stilling som verks...,201501,Jobbnorge ID: 108301<br><br><p dir=#LTR# align...
3,101201412000013,101,23.12.2014,23.01.2015,201501,1,Annonsert i media,01,Østfold,0101,...,Butikk- og salgsarbeid,3322,Selgere (engros),Key Account Manager (salg),886332882,SMART INNOVATION NORWAY AS,1,Key Account Manager,201501,# har ambisjoner om å vokse og styrker teamet ...
4,101201412000014,101,23.12.2014,30.01.2015,201501,1,Annonsert i media,01,Østfold,0101,...,Ingeniør- og ikt-fag,2152,Sivilingeniører (elektronikk),Forsker (elektronikk),886332882,SMART INNOVATION NORWAY AS,1,Senior Reasearchers/Reasearchers,201501,# is now expanding its leading team with senio...


In [13]:
whitespace_pattern = re.compile(r'\s+')

def remove_tags(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def unescape_html(raw_html):
    return html.unescape(raw_html)

def remove_whitespace(text):
    return whitespace_pattern.sub(' ', text)

# Select subset of columns
col = [
    'stilling_kilde', 
    'stillingsnummer', 
    'stillingsbeskrivelse', 
    'yrke_grovgruppe',
    'arbeidssted_fylkesnummer',
    'arbeidssted_kommunenummer',
    'virksomhet_navn',
    'arbeidssted_fylke',
]
df2 = df[col]


# Remove html tags from description
df2[['stillingsbeskrivelse']] = df['stillingsbeskrivelse'].map(remove_tags).map(unescape_html).map(remove_whitespace)
df2['is_from_nav'] = df['stilling_kilde'].map(lambda x: x=='Reg av arb.giver på nav.no')
df2['from_media'] = df['stilling_kilde'].map(lambda x: x=='Annonsert i media')
df2['is_healthcare'] = df['yrke_grovgruppe'].map(lambda x: x=='Helse, pleie og omsorg')
df2['is_industrial'] = df['yrke_grovgruppe'].map(lambda x: x=='Industriarbeid')
df2['contains_nav'] = df['stillingsbeskrivelse'].map(lambda x: 'nav' in x)

print(df2.shape)
df2.head()




/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing

(118096, 13)


/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn,arbeidssted_fylke,is_from_nav,from_media,is_healthcare,is_industrial,contains_nav
0,Annonsert i media,101201412000010,# senter for framandspråk i opplæringa er det ...,Undervisning,01,0101,HØGSKOLEN I ØSTFOLD,Østfold,False,True,False,False,False
1,Overført fra arbeidsgiver,101201412000011,Jobbnorge ID: 108617# fengsel er det ledig fas...,Serviceyrker og annet arbeid,01,0101,HALDEN FENGSEL,Østfold,False,False,False,False,False
2,Overført fra arbeidsgiver,101201412000012,Jobbnorge ID: 108301# fengsel er det ledig fas...,Serviceyrker og annet arbeid,01,0101,HALDEN FENGSEL,Østfold,False,False,False,False,False
3,Annonsert i media,101201412000013,# har ambisjoner om å vokse og styrker teamet ...,Butikk- og salgsarbeid,01,0101,SMART INNOVATION NORWAY AS,Østfold,False,True,False,False,True
4,Annonsert i media,101201412000014,# is now expanding its leading team with senio...,Ingeniør- og ikt-fag,01,0101,SMART INNOVATION NORWAY AS,Østfold,False,True,False,False,True


In [94]:
from sklearn.model_selection import train_test_split

data_subset_size = len(df2) # Just use a small dataset when experimenting or it will take too much time
target_column_name = 'yrke_grovgruppe' # Which label to predict
random_seed = 305

data_sample = df2.sample(n=data_subset_size, random_state=random_seed)

# Make training and test sets of the data. 
X_train, X_test, y_train, y_test = train_test_split(
    data_sample, 
    data_sample[target_column_name],
    test_size=.2,
    random_state=random_seed
)

X_train

,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn,arbeidssted_fylke,is_from_nav,from_media,is_healthcare,is_industrial,contains_nav
67146,Reg av arb.giver på nav.no,1130201510000017,På grunn av svangerskapspermisjon søker vi ett...,Undervisning,11,1130,MINIKLUBBEN NATURBARNEHAGE AS,Rogaland,True,False,False,False,False
57610,Reg av arb.giver på nav.no,901201508000006,Vi trenger nye medarbeidere! Har du lyst til o...,Reiseliv og transport,09,0901,BASO AS,Aust-Agder,True,False,False,False,False
101639,Annonsert i media,1719201507000024,"Medisinsk avdeling, # Medisinsk klinikk ved # ...","Helse, pleie og omsorg",17,1719,HELSE NORD-TRØNDELAG HF,Nord-Trøndelag,False,True,True,False,False
94975,Reg av arb.giver på nav.no,1603201505000330,# søker trivelig resepsjonist - Vi søker somme...,Kontorarbeid,16,1601,THON HOTEL PRINSEN,Sør-Trøndelag,True,False,False,False,False
110150,Overført fra arbeidsgiver,1902201502000256,Jobbnorge ID: 110259 1-100% stilling som rådgi...,Akademiske yrker,19,1902,SENTRALADMINISTRASJONEN,Troms,False,False,False,False,False
11943,Annonsert i media,230201510000057,Barn leger Vi søker etter barnelege med erfari...,"Helse, pleie og omsorg",02,0230,AHUS NORDBYHAGEN,Akershus,False,True,True,False,True
77809,Annonsert i media,1209201507000270,"Assistent - #, seksjon Ytre område Hospitaldri...",Serviceyrker og annet arbeid,12,1201,HELSE BERGEN HF,Hordaland,False,True,False,False,False
31419,Reg av arb.giver på nav.no,330201504000008,Pedagogisk leder søkes til Barnehagen#.Vi søke...,Undervisning,03,0301,BARNEHAGENVÅR,Oslo,True,False,False,False,False
92436,Annonsert i media,1547201510000003,Sjukepleiar 100 % vikariat - Eining institusjo...,"Helse, pleie og omsorg",15,1547,AUKRA KOMMUNE HEIMETENESTA ÅPEN,Møre og Romsdal,False,True,True,False,False
82874,Annonsert i media,1247201501000027,Vi har ledig stilling som: Lastebilmekaniker ...,Industriarbeid,12,1247,ØLEN BETONG GRUPPEN AS,Hordaland,False,True,False,True,False


In [95]:
# Verify that the data looks as expected
X_test

,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn,arbeidssted_fylke,is_from_nav,from_media,is_healthcare,is_industrial,contains_nav
52098,Annonsert i media,704201508000120,? Ref 15/64Det er et ledig vikariat i 100% sti...,Akademiske yrker,06,0604,HØGSKOLEN I SØRØST-NORGE,Buskerud,False,True,False,False,True
34798,Annonsert i media,334201509000044,Flymedisinsk seksjon er lokalisert i Oslo hvor...,Akademiske yrker,03,0301,LUFTFARTSTILSYNET,Oslo,False,True,False,False,False
70843,Annonsert i media,1164201412000047,Selger/prosjektutvikler for rehabiliteringsopp...,Butikk- og salgsarbeid,11,1103,JADARHUS BYGG AS,Rogaland,False,True,False,False,False
8802,Annonsert i media,219201509000202,Sykepleier til fast stilling - Bemanningsenhet...,"Helse, pleie og omsorg",02,0219,VESTRE VIKEN HF,Akershus,False,True,True,False,False
21791,Annonsert i media,315201501000102,Mediesjef Vi søker en kunnskapsrik og inspirer...,Ledere,03,0301,REDD BARNA,Oslo,False,True,False,False,False
14980,Annonsert i media,238201508000006,Nannestad ungdomsskole har ledig 89 % stilling...,Barne- og ungdomsarbeid,02,0238,NANNESTAD UNGDOMSSKOLE,Akershus,False,True,False,False,True
34468,Annonsert i media,334201505000001,Dagrehabiliteringsavdelingen er et tverrfaglig...,"Helse, pleie og omsorg",03,0301,BYDELSADMINISTRASJONEN,Oslo,False,True,True,False,False
24406,Annonsert i media,316201504000016,Turnus- og vaktansvarlig - Kvinne- og barnekli...,"Helse, pleie og omsorg",03,0301,OSLO UNIVERSITETSSYKEHUS HF,Oslo,False,True,True,False,False
51008,Reg av arb.giver på nav.no,701201507000008,Vi utvider og søker etter flere terapeuter til...,"Helse, pleie og omsorg",07,0701,FAMILIE OG AKUPUNKTURKLINIKKEN AS,Vestfold,True,False,True,False,False
25604,Reg av arb.giver på nav.no,316201507000013,# & Bar søker servitører på hel - og deltid Sø...,Reiseliv og transport,03,0301,VENNER MAT & VINHUS AS,Oslo,True,False,False,False,True


In [96]:
# Make the pipeline that transforms our data and trains the classifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_df=.25)),
    ('svc', LinearSVC()),
])

pipeline.fit(X_train.stillingsbeskrivelse.values, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.25, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [97]:
# Test the data by running the test data set on the trained pipeline
predicted = pipeline.predict(X_test.stillingsbeskrivelse.values)
pprint(predicted)
np.mean(predicted == y_test)

array(['Akademiske yrker', 'Akademiske yrker', 'Butikk- og salgsarbeid',
       ..., 'Akademiske yrker', 'Undervisning', 'Undervisning'],
      dtype=object)


0.8265876375952582

In [98]:
from sklearn import metrics

# Print some stats of our results
print(metrics.classification_report(y_test, predicted))
print(metrics.accuracy_score(y_test, predicted))

                                    precision    recall  f1-score   support

                  Akademiske yrker       0.74      0.77      0.76      1725
           Barne- og ungdomsarbeid       0.80      0.71      0.75       804
            Butikk- og salgsarbeid       0.85      0.88      0.87      1728
                    Bygg og anlegg       0.82      0.81      0.81      1122
            Helse, pleie og omsorg       0.90      0.95      0.92      6621
                    Industriarbeid       0.72      0.73      0.72       832
Ingen yrkesbakgrunn eller uoppgitt       0.39      0.12      0.18        75
              Ingeniør- og ikt-fag       0.76      0.79      0.77      1785
       Jordbruk, skogbruk og fiske       0.77      0.59      0.67       159
                      Kontorarbeid       0.72      0.62      0.67       963
                            Ledere       0.58      0.53      0.55       601
            Meglere og konsulenter       0.69      0.64      0.66       883
           

In [101]:
# Try the pipeline with some custom data

prediction = pipeline.predict(['trenger noen som er snille og greie. helst med mye driv'])
pprint(prediction)

array(['Reiseliv og transport'], dtype=object)


In [79]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

def evaluate_svm(train_data, train_labels, test_data, test_labels, n_jobs=-1):
    """
    Evaluates a representation using a Linear SVM
    It uses 3-fold cross validation for selecting the C parameter
    :param train_data:
    :param train_labels:
    :param test_data:
    :param test_labels:
    :param n_jobs:
    :return: the test accuracy
    """
    
    parameters = {
        'tfidf__max_df': (0.25,),
        #'vect__max_features': (None, 5000, 10000, 50000),
        #'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
        #'tfidf__use_idf': (True, False),
        'tfidf__norm': ('l1', 'l2'),
        'svc__C': (1,),
        #'clf__penalty': ('l2', 'elasticnet'),
        #'svc__n_iter': (10, 50, 80),
    }

    grid_search = GridSearchCV(pipeline, parameters, n_jobs=n_jobs, cv=3)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    
    grid_search.fit(train_data, train_labels)
    
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    lin_svm_test = grid_search.score(test_data, test_labels)
    return lin_svm_test 

evaluate_svm(X_train.stillingsbeskrivelse.values, y_train, X_test.stillingsbeskrivelse.values, y_test)

Performing grid search...
pipeline: ['tfidf', 'svc']
parameters:
{'svc__C': (1,), 'tfidf__max_df': (0.25,), 'tfidf__norm': ('l1', 'l2')}
done in 12.249s

Best score: 0.936
Best parameters set:
	svc__C: 1
	tfidf__max_df: 0.25
	tfidf__norm: 'l2'


0.9355

In [75]:
LinearSVC().get_params().keys()

dict_keys(['max_iter', 'loss', 'dual', 'random_state', 'C', 'multi_class', 'fit_intercept', 'tol', 'intercept_scaling', 'verbose', 'penalty', 'class_weight'])

In [16]:
# Analyze which words are most characteristic of each category

tfidf = TfidfVectorizer()

features = tfidf.fit_transform(df2.stillingsbeskrivelse)
labels = df2.yrke_grovgruppe
features.shape

from sklearn.feature_selection import chi2
N = 10
for label in labels.drop_duplicates():
    features_chi2 = chi2(features, labels == label)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    print("# '{}':".format(label))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))


# 'Undervisning':
  . Most correlated unigrams:
. lærer
. undervisning
. barnehagen
. barnehagelærer
. barnehage
. elever
. rektor
. skole
. skolen
. pedagogisk
# 'Serviceyrker og annet arbeid':
  . Most correlated unigrams:
. politistasjon
. kirke
. sko
. etterforskning
. politioverbetjent
. renhold
. frisør
. renholder
. politidistrikt
. politibetjent
# 'Butikk- og salgsarbeid':
  . Most correlated unigrams:
. kundebehandling
. selge
. du
. butikkens
. butikkmedarbeider
. butikken
. butikksjef
. selger
. butikk
. salg
# 'Ingeniør- og ikt-fag':
  . Most correlated unigrams:
. ikt
. in
. of
. sivilingeniør
. teknisk
. prosjektering
. it
. bioingeniør
. ingeniør
. and
# 'Bygg og anlegg':
  . Most correlated unigrams:
. elektrikere
. svennebrev
. adk
. maskinfører
. snekker
. fagbrev
. tømrere
. rørlegger
. elektriker
. tømrer
# 'Helse, pleie og omsorg':
  . Most correlated unigrams:
. lege
. sykehjem
. helg
. vernepleier
. sykepleiere
. helsefagarbeider
. turnus
. autorisasjon
. pasient

In [19]:
import csv
non_alphanumeric_pattern = re.compile(r'\W')

out = df2[:][['is_healthcare', 'stillingsbeskrivelse']]
#out['is_healthcare'] = out['is_healthcare'].map(lambda x: x.replace(' ', '_'))
out['is_healthcare'] = out['is_healthcare'].map(lambda x: '__label__' + str(x))
out['stillingsbeskrivelse'] = out['stillingsbeskrivelse'].map(lambda x: non_alphanumeric_pattern.sub(' ', x))
out['stillingsbeskrivelse'] = out['stillingsbeskrivelse'].map(lambda x: whitespace_pattern.sub(' ', x))
out['stillingsbeskrivelse'] = out['stillingsbeskrivelse'].map(lambda x: x.lower())
out.to_csv(os.path.join('.', 'fasttext', 'out.txt'), escapechar=' ', sep=' ', header=None, mode='w', index=False, quoting=csv.QUOTE_NONE)
out

,is_healthcare,stillingsbeskrivelse
0,__label__False,pleie
1,__label__False,jobbnorge id 108617 fengsel er det ledig fast ...
2,__label__False,jobbnorge id 108301 fengsel er det ledig fast ...
3,__label__False,har ambisjoner om å vokse og styrker teamet v...
4,__label__False,is now expanding its leading team with senior...
5,__label__False,arbeidsoppgaver utføre ettersyn og kontroll sa...
6,__label__False,selskapet er i sterk vekst og ønsker derfor å ...
7,__label__False,for vår kunde søker vi dyktig formann til innl...
8,__label__False,caverion i søker dyktige kuldemontører teknike...
9,__label__False,øst as ønsker å styrke utvikle og bygge opp vå...


# Results from fasttext in the same dataset as above

[Fasttext website](https://fasttext.cc)
[Fasttext research paper](https://arxiv.org/pdf/1607.01759.pdf)


Fasttext is an alternative to bag-of-words models used in sklearn. We did not get as good results as linear SVM for multiclass classification, but not that far off. (~0.76 vs ~0.80)


```
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext supervised -input out.train -output model -epoch 25
Read 21M words
Number of words:  250574
Number of labels: 15
Progress: 100.0%  words/sec/thread: 2777322  lr: 0.000000  loss: 0.489928  eta: 0h0m h-14m 
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext test model.bin out.valid 1N	20000
P@1	0.765
R@1	0.765
Number of examples: 20000
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext supervised -input out.train -output model -epoch 25 -lr 1.0
Read 21M words
Number of words:  250574
Number of labels: 15
Progress: 100.0%  words/sec/thread: 2705054  lr: 0.000000  loss: 0.415153  eta: 0h0m 14m 
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext test model.bin out.valid 1N	20000
P@1	0.764
R@1	0.764
Number of examples: 20000
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext test model.bin out.valid 2
N	20000
P@2	0.431
R@2	0.861
Number of examples: 20000
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext test model.bin out.valid 1
N	20000
P@1	0.764
R@1	0.764
Number of examples: 20000
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext supervised -input out.train -output model -epoch 25 -lr 1.0 -bucket 200000 -loss hs
Read 21M words
Number of words:  250574
Number of labels: 15
Progress: 100.0%  words/sec/thread: 2916374  lr: 0.000000  loss: 0.509901  eta: 0h0m -14m 
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext test model.bin out.valid 1N	20000
P@1	0.747
R@1	0.747
Number of examples: 20000
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext supervised -input out.train -output model -epoch 25 -lr 1.0 -bucket 200000 -loss hs -dim 50
Read 21M words
Number of words:  250574
Number of labels: 15
Progress: 100.0%  words/sec/thread: 3454463  lr: 0.000000  loss: 0.498687  eta: 0h0m -14m 
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext test model.bin out.valid 1N	20000
P@1	0.744
R@1	0.744
Number of examples: 20000
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext supervised -input out.train -output model -epoch 40 -lr 0.5
Read 21M words
Number of words:  250574
Number of labels: 15
Progress: 100.0%  words/sec/thread: 2749872  lr: 0.000000  loss: 0.354190  eta: 0h0m -14m 
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext test model.bin out.valid 1N	20000
P@1	0.76
R@1	0.76
Number of examples: 20000
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext supervised -input out.train -output model -epoch 40 -lr 0.2
Read 21M words
Number of words:  250574
Number of labels: 15
Progress: 100.0%  words/sec/thread: 2752216  lr: 0.000000  loss: 0.359838  eta: 0h0m -14m 
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ls
cooking.stackexchange.id      cooking.stackexchange.txt  model.bin  out.train  out.valid   v0.1.0.zip
cooking.stackexchange.tar.gz  fastText-0.1.0		 model.vec  out.txt    readme.txt
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# ./fastText-0.1.0/fasttext test model.bin out.valid 1N	20000
P@1	0.762
R@1	0.762
Number of examples: 20000
(base) root@e69f4f8ed09b:/opt/notebooks/fasttext# 

```